#### 0. Import basic methods from source.py
Import useful methods from source.py (NB: can later be integrated into a package)

In [1]:
from source import *

#### 1. Open xls file and ingest Inputs table
Note I am assuming that all inputs will be aggregated into one single inputs table

In [2]:
filename = r'H2 Model Wouter.xlsm'

In [ ]:
%%time
# load workbook
wb = load_workbook(filename)

C:\Users\MRV\Anaconda3\envs\gdal\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# load worksheet
ws = wb["Input Tab"]

In [ ]:
mapping = {}

for entry, data_boundary in ws.tables.items():
    # parse the data within the ref boundary
    data = ws[data_boundary]
    
    # extract the data (the inner list comprehension gets the values for each cell in the table)
    content = [[cell.value for cell in ent] for ent in data]
    
    # find header
    header = content[0]
    
    # find the rest ... the contents, excluding the header
    rest = content[1:]
    
    # create dataframe with the column names
    # and pair table name with dataframe
    df = pd.DataFrame(rest, columns = header)
    mapping[entry] = df

In [ ]:
Inputs = pd.DataFrame(mapping['Scenario1'])
Inputs.head()

#### 2. Data inspection 

In [ ]:
Inputs.columns

In [ ]:
Inputs[Inputs['Sub-system']=='Wind energy source & Transport'].head()

In [ ]:
list(Inputs['Sub-system'].unique())

In [ ]:
list(Inputs['Element'].unique())

In [ ]:
list(Inputs['Component'].unique())

#### 3. Select one item from the Inputs table (generate cashflow overview)

##### A. do per item

In [ ]:
%%time
df_Foundations = Inputs_2_cashflow(Inputs, 
                                   startyear=2023,
                                   lifecycle=50,
                                   subsystem='Wind energy source & Transport',
                                   element='Offshore wind park',
                                   component='Foundation & cable',
                                   Debug=True)
df_Foundations.head()

In [ ]:
%%time
df_npv_Foundations = calculate_npv(df_Foundations)
df_npv_Foundations.head()

In [ ]:
%%time
lims = [-3500, 3500]
create_npv_plot(df_npv_Foundations, cash_flow_lims=lims, npv_lims=lims)

In [ ]:
%%time
df_Turbines = Inputs_2_cashflow(Inputs, 
                                   startyear=2023,
                                   lifecycle=11,
                                   subsystem='Wind energy source & Transport',
                                   element='Offshore wind park',
                                   component='Turbine',
                                   Debug=True)
df_Turbines.head()

In [ ]:
%%time
df_npv_Turbines = calculate_npv(df_Turbines)
df_npv_Turbines.head()

In [ ]:
%%time
lims = [-3500, 3500]
create_npv_plot(df_npv_Turbines, cash_flow_lims=lims, npv_lims=lims)

##### B. combine two items

In [ ]:
df_Foundations = Inputs_2_cashflow(Inputs, startyear=2023, lifecycle=11, component='Foundation & cable',Debug=False)
df_Turbines = Inputs_2_cashflow(Inputs, startyear=2023, lifecycle=11, component='Turbine',Debug=False)
df_combined = combine_cashflow_dataframes([df_Foundations, df_Turbines])

In [ ]:
df_npv_combined = calculate_npv(df_combined)

In [ ]:
%%time
lims = [-4500, 4500]
create_npv_plot(df_npv_combined, cash_flow_lims=lims, npv_lims=lims)